In [1]:
import numpy as np

import itertools

In [2]:
def left(x, y):
    return x-1, y

def right(x, y):
    return x+1, y

def up(x, y):
    return x, y + 1

def down(x, y):
    return x, y - 1

def stay(x, y):
    return x, y

In [3]:
class Cell(object):
    def __init__(self, actions, default_value = 0.):
        self.actions = {a:0. for a in actions}
        self.update_policy()
        self.value = default_value

    def update_policy(self):
        self.policy = max(self.actions.items(), key = lambda p: p[1])[0]


class Grid(object):
    def __init__(self, dim=10, default_value=0.):
        self.dim = dim
        
        self.data = np.array([[None for _ in range(dim)] for _ in range(dim)])
        
        for x in range(1, dim - 1):
            for y in range(1, dim - 1):
                self.data[x][y] = Cell([left, right, up, down], default_value=default_value)
                
        for x in range(1, dim - 1):
            self.data[x][0] = Cell([left, right, up], default_value=default_value)
            self.data[x][dim - 1] = Cell([left, right, down], default_value=default_value)
            
        for y in range(1, dim - 1):
            self.data[0][y] = Cell([right, up, down], default_value=default_value)
            self.data[dim - 1][y] = Cell([left, up, down], default_value=default_value)
            
        self.data[0][0] = Cell([up, right], default_value=default_value)
        self.data[dim-1][dim-1] = Cell([stay], default_value=100.)
        self.data[0][dim - 1] = Cell([right, down], default_value=default_value)
        self.data[dim - 1][0] = Cell([left, up], default_value=default_value)
        
        self.data[dim - 1][dim - 2].actions[up] = 100.
        self.data[dim - 2][dim - 1].actions[right] = 100.
        
        self.data[dim - 1][dim - 2].update_policy()
        self.data[dim - 2][dim - 1].update_policy()
        
    def best_path(self):
        path = list()
        x, y = 0, 0
                
        while not(x == self.dim - 1 and y == self.dim - 1):
            path.append((x, y))
            x, y = self.data[x, y].policy(x, y)
        
        path.append((x, y))
        return path
                
        

In [4]:
g = Grid(10)
theta, disc = 1., 0.8

In [5]:
def iter_values(grid, theta, disc):
    def eval():
        delta = -1.
        for x in range(grid.dim):
            for y in range(grid.dim):
                c = grid.data[x, y]
                
                prev, act = c.value, c.policy
                c.value = c.actions[act] + disc*grid.data[act(x, y)].value
                delta = max(prev, abs(prev - c.value))
                
        return delta
            
    
    n_iters = 0
    
    while eval() >= theta:
        n_iters += 1
        
    return n_iters


def iter_policies(grid, theta, disc):     
    improving = True
    n_iters = 0

    while improving:
        improving = False
        iter_values(grid, theta, disc)
        
        for x in range(grid.dim):
            for y in range(grid.dim):
                c = grid.data[x, y]
                best_policy = c.policy
                
                delta = -100500.
                for act, reward in c.actions.items():
                    q = reward + disc*grid.data[act(x, y)].value
                    
                    if q > delta:
                        delta, best_policy = q, act
                
                if best_policy != c.policy:
                    c.policy, improving = best_policy, True
    
        n_iters += 1
    
    return n_iters

In [6]:
iter_policies(g, theta, disc)

17

In [7]:
g.best_path()

[(0, 0),
 (0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (9, 2),
 (9, 3),
 (9, 4),
 (9, 5),
 (9, 6),
 (9, 7),
 (9, 8),
 (9, 9)]